In [1]:
# 필요한 라이브러리 불러오기
import tensorflow as tf
import numpy as np
import seaborn as sns

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

# Tensorflow
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#from keras_tuner.tuners import RandomSearch

import scikitplot as skplt
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt

In [2]:
img_width = 512
img_height = 512

In [3]:
CurrentDirectory = "./dataset/"

train_directory = CurrentDirectory + 'train/'
test_directory = CurrentDirectory + 'test/'
model_directory = './MODEL/'
tensorboard_directory = './Tensorboard/'

In [5]:
VGG16Model = tf.keras.applications.VGG19(
    input_shape=(img_width,img_height,3),
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None)
x = MaxPooling2D()(VGG16Model.output)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation='softmax', name = "output_node")(x)
# Input ~ Output 연결해주기
model = Model(inputs=VGG16Model.input, outputs=predictions)

In [7]:
model = load_model('./MODEL/VGG19-051-0.0323-0.9943.hdf5')

In [8]:
def preprocessing_image(img):
    resize_img = cv2.resize(img, (512,512))
    img_array_expanded_dims = np.expand_dims(resize_img, axis=0)
    return tf.keras.applications.vgg19.preprocess_input(img_array_expanded_dims)

In [9]:
file_list = os.listdir('./dataset/test/')

In [10]:
result = []
for i in range(len(file_list)):
    image = cv2.imread('./dataset/test/'+file_list[i])
    p_image = preprocessing_image(image)
    prediction = model.predict(p_image)[0]
    pred = prediction.argmax(axis=-1)
    # if pred == 0:
    #     pred = 8
    # elif pred == 1:
    #     pred = 5
    # elif pred == 2:
    #     pred = 4
    # elif pred == 3:
    #     pred = 9
    # elif pred == 4:
    #     pred = 1
    # elif pred == 5:
    #     pred = 7
    # elif pred == 6:
    #     pred = 6
    # elif pred == 7:
    #     pred = 3
    # elif pred == 8:
    #     pred = 2
    # else:
    #     pred = 0
    result.append(pred)

1/1 [==============================] - 0s 12ms/step


In [11]:
print(result)

[7, 1, 9, 7, 6, 1, 3, 6, 3, 1, 0, 1, 7, 1, 2, 3, 2, 0, 4, 0, 6, 4, 8, 7, 8, 9, 0, 9, 1, 8, 0, 8, 6, 3, 3, 0, 7, 8, 9, 0, 5, 4, 3, 8, 5, 6, 4, 5, 0, 3, 5, 4, 1, 5, 9, 3, 5, 6, 4, 0, 0, 6, 1, 5, 0, 4, 0, 0, 0, 0, 3, 4, 4, 4, 6, 4, 4, 0, 7, 5, 7, 9, 9, 4, 3, 0, 1, 4, 1, 8, 9, 0, 7, 6, 0, 5, 0, 6, 0, 7, 3, 5, 0, 9, 7, 6, 5, 9, 7, 0, 7, 8, 4, 8, 8, 5, 4, 5, 1, 5, 4, 5, 1, 6, 4, 0, 9, 0, 6, 0, 8, 7, 1, 8, 6, 3, 4, 6, 7, 7, 1, 0, 0, 5, 7, 6, 2, 6, 7, 6, 4, 3, 5, 7, 8, 5, 0, 6, 6, 5, 6, 0, 3, 0, 5, 9, 4, 0, 5, 1, 9, 7, 1, 4, 3, 3, 3, 0, 1, 4, 1, 4, 6, 7, 3, 9, 3, 0, 7, 0, 0, 7, 0, 8, 0, 1, 1, 9, 0]


In [12]:
test_df = pd.read_csv('./sample_submission.csv')
test_df.head()

,file_name,label
0,001.PNG,0
1,002.PNG,0
2,003.PNG,0
3,004.PNG,0
4,005.PNG,0


In [13]:
test_df.label = result
test_df

,file_name,label
0,001.PNG,7
1,002.PNG,1
2,003.PNG,9
3,004.PNG,7
4,005.PNG,6
...,...,...
194,195.PNG,0
195,196.PNG,1
196,197.PNG,1
197,198.PNG,9


In [14]:
test_df.to_csv('v19_max.csv',index=False)